# Generating a CIFF file from a Terrier index

This notebook demostrates generating a CIFF file from a Terrier index, through PyTerrier.

## Setup

In [1]:
!pip install python-terrier

     |████████████████████████████████| 81kB 3.1MB/s 
     |████████████████████████████████| 1.1MB 6.1MB/s 
     |████████████████████████████████| 71kB 4.4MB/s 
     |████████████████████████████████| 163kB 14.0MB/s 
     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 645kB 13.8MB/s 
     |████████████████████████████████| 133kB 20.0MB/s 
     |████████████████████████████████| 5.5MB 19.0MB/s 
     |████████████████████████████████| 1.8MB 28.1MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.5.0-cp37-none-any.whl size=79538 sha256=0e0aaa5e1a597387e9ea4f53d4917791a3f5582c10efe5f4c4c3f1f2e5cb0496
  Stored in directory: /root/.cache/pip/wheels/b4/4d/75/ab103d23ae843881ff3adb2d75a5b8d624e22adade8d5f7506
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=53483b17d932f0735fede8e446fc4521b2f29ca08f4a8527f6c66f74c53c1bb9
  Stored in directory: /root/.c

This downloads the CIFF definition from the CIFF repostiory.

In [2]:
!wget https://raw.githubusercontent.com/osirrc/ciff/master/src/main/protobuf/CommonIndexFileFormat.proto

--2021-03-29 11:28:29--  https://raw.githubusercontent.com/osirrc/ciff/master/src/main/protobuf/CommonIndexFileFormat.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2313 (2.3K) [text/plain]
Saving to: ‘CommonIndexFileFormat.proto’

CommonIndexFileForm 100%[===================>]   2.26K  --.-KB/s    in 0s      

2021-03-29 11:28:30 (34.4 MB/s) - ‘CommonIndexFileFormat.proto’ saved [2313/2313]



Make the python code for CIFF

In [3]:
!mkdir -p ciff
!protoc --python_out=ciff ./CommonIndexFileFormat.proto

Start PyTerrier. Ensure that Terrier has imported the CIFF plugin. 

In [4]:
import pyterrier as pt
pt.init(packages=["com.github.terrierteam:terrier-ciff:-SNAPSHOT"])

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


Lets use PyTerrier's prebuilt index for the small Vaswani test collection (~11k abstracts).

In [5]:
dataset = pt.get_dataset("vaswani")
index = pt.IndexFactory.of(dataset.get_index())

## Generating the CIFF file

In [7]:
from ciff.CommonIndexFileFormat_pb2 import Header, PostingsList, Posting, DocRecord

In [20]:
!rm -f ./ciff.vaswani.gz
file = pt.io.autoopen("./ciff.vaswani.gz", "wb")

In [21]:
def write_delimited(f, o):
  from google.protobuf.internal.encoder import _VarintBytes
  from google.protobuf.internal.decoder import _DecodeVarint32
  size = o.ByteSize()
  f.write(_VarintBytes(size))
  f.write(o.SerializeToString())


In [22]:
header = Header()
header.version = 1
header.num_postings_lists = index.getCollectionStatistics().getNumberOfUniqueTerms()
header.num_docs = index.getCollectionStatistics().getNumberOfDocuments()
header.total_docs = index.getCollectionStatistics().getNumberOfDocuments()
header.total_postings_lists = index.getCollectionStatistics().getNumberOfUniqueTerms()
header.total_terms_in_collection = index.getCollectionStatistics().getNumberOfTokens()
header.average_doclength  = index.getCollectionStatistics().getAverageDocumentLength()
header.description = index.getIndexRef().toString()
write_delimited(file, header)

In [23]:
lexicon = index.getLexicon()
inv = index.getInvertedIndex()
for kv in pt.tqdm(lexicon, unit="t"):
  pl = PostingsList()
  pl.term = kv.getKey()
  pl.df = kv.getValue().getDocumentFrequency()
  pl.cf = kv.getValue().getFrequency()
  prevDocid = -1;
  for p in inv.getPostings(kv.getValue()):
    posting = Posting()
    curDocid = p.getId()
    code = curDocid if  prevDocid == -1 else curDocid - prevDocid;
    posting.docid = code
    posting.tf = p.getFrequency()
    pl.postings.append(posting)
    prevDocid = curDocid

  write_delimited(file, pl)

In [24]:
metaindex = index.getMetaIndex()
docindex = index.getDocumentIndex()
for docid in pt.tqdm(range(0, index.getCollectionStatistics().getNumberOfDocuments()), unit="d"):
  doc = DocRecord()
  doc.docid = docid
  doc.collection_docid = metaindex.getItem("docno", docid)
  doc.doclength = docindex.getDocumentLength(docid)
  write_delimited(file, doc)

In [25]:
file.close()

## Making a Terrier index from the CIFF file

Here we use the terrier-ciff plugin for Terier.

In [26]:
!mkdir -p from_ciff_index
pt.run("ciff-ingest", ["-I", "./from_ciff_index/data.properties", "./ciff.vaswani.gz"])

Finished ingesting ./ciff.vaswani.gz (11429 documents)
New index at /content/from_ciff_index//data.properties


## Validation Effectiveness of the CIFF index vs the original CIFF index.

In [27]:
pt.Experiment(
    [pt.BatchRetrieve(index), pt.BatchRetrieve( "./from_ciff_index/data.properties")],
    dataset.get_topics(),
    dataset.get_qrels(),
    ["map"]
)

,name,map
0,BR(DPH),0.283621
1,BR(DPH),0.283621
